<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/4_hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-14 19:12:24--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-14 19:12:24 (105 MB/s) - ‘utils.py’ saved [11733/11733]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [ ]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [8]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model(img_size=IMG_SIZE)

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [9]:
callbacks = hsv_creator.get_callbacks()

In [10]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        class_weight=class_weight)

Epoch 1/25
533/533 [==============================] - 1674s 3s/step - loss: 123.0426 - auc: 0.4906 - accuracy: 0.5832
Epoch 2/25
533/533 [==============================] - 154s 286ms/step - loss: 76.1834 - auc: 0.6381 - accuracy: 0.7168
Epoch 3/25
533/533 [==============================] - 153s 284ms/step - loss: 61.5543 - auc: 0.6960 - accuracy: 0.7608
Epoch 4/25
533/533 [==============================] - 153s 285ms/step - loss: 55.7740 - auc: 0.7220 - accuracy: 0.7808
Epoch 5/25
533/533 [==============================] - 153s 285ms/step - loss: 45.2228 - auc: 0.7505 - accuracy: 0.8030
Epoch 6/25
533/533 [==============================] - 153s 284ms/step - loss: 41.7660 - auc: 0.7622 - accuracy: 0.8138
Epoch 7/25
533/533 [==============================] - 153s 285ms/step - loss: 32.6031 - auc: 0.7976 - accuracy: 0.8372
Epoch 8/25
533/533 [==============================] - 153s 285ms/step - loss: 28.3173 - auc: 0.8133 - accuracy: 0.8450
Epoch 9/25
533/533 [=============================

In [11]:
!cp ./logs/checkpoints/HSV-model-09.h5 ./drive/MyDrive/checkpoints/hsv-model-09.h5
!cp ./logs/checkpoints/HSV-model-15.h5 ./drive/MyDrive/checkpoints/hsv-model-15.h5
!cp ./logs/checkpoints/HSV-model-20.h5 ./drive/MyDrive/checkpoints/hsv-model-20.h5
!cp ./logs/checkpoints/HSV-model-25.h5 ./drive/MyDrive/checkpoints/hsv-model-25.h5

In [12]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=30,
                          class_weight=class_weight,
                          initial_epoch=25)

Epoch 26/30
533/533 [==============================] - 154s 285ms/step - loss: 1.8893 - auc: 0.9818 - accuracy: 0.9664
Epoch 27/30
533/533 [==============================] - 152s 283ms/step - loss: 1.8864 - auc: 0.9796 - accuracy: 0.9651
Epoch 28/30
533/533 [==============================] - 153s 285ms/step - loss: 1.6552 - auc: 0.9843 - accuracy: 0.9701
Epoch 29/30
533/533 [==============================] - 153s 285ms/step - loss: 1.4649 - auc: 0.9859 - accuracy: 0.9725
Epoch 30/30
533/533 [==============================] - 153s 285ms/step - loss: 1.3162 - auc: 0.9876 - accuracy: 0.9734


In [13]:
!cp ./logs/checkpoints/HSV-model-30.h5 ./drive/MyDrive/checkpoints/hsv-model-30.h5